In [128]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import random
import pandas as pd
import time
import datetime

In [129]:
class VacumAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        if self.model.is_dirt(self.pos):
            self.model.clean_dirt(self.pos)
            self.move()
        else:
            # Move to a random neighbor cell, but check first if in that cell there isn't another agent
            self.move()

In [130]:
class VacumModel(Model):
    def __init__(self,width,height, num_agents, max_steps, dirty_cells):
        self.num_agents = num_agents
        self.max_steps = max_steps
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, torus=False)
        self.steps = 0
        self.dirty_cells = dirty_cells
        self.dirt = np.zeros((width, height))

        # Calculate the number of dirty cells
        num_dirty_cells = int(dirty_cells * width * height)

        # Create a list of dirty cells
        matrix = [(x,y) for x in range(width) for y in range(height)]
        random.shuffle(matrix)
        dirty_cells = matrix[:num_dirty_cells]
        for cell in dirty_cells:
            self.dirt[cell] = 1

        # Create agents
        for i in range(num_agents):
            a = VacumAgent(i, self)
            self.schedule.add(a) #########
            self.grid.place_agent(a, (1, 1))

    def count_dirt(self):
        # Count the number of dirty cells
        return sum(cell == 1 for cell in self.dirt.flatten())
    

    def is_dirt(self, pos):
        return self.dirt[pos[0], pos[1]] == 1


    def clean_dirt(self, pos):
        self.dirt[pos[0]][pos[1]] = 0

    def step(self):
        self.schedule.step()
        self.steps += 1

In [131]:
def run_model(width, height, num_agents, max_steps, dirty_cells):
    model = VacumModel(width, height, num_agents, max_steps, dirty_cells)
    for step in range(max_steps):
        model.step()
        if np.all(model.dirt == 0):
            return (
                step,
                (1 - np.mean(model.dirt)) * 100
            )
    return (
        max_steps,
        (1 - np.mean(model.dirt)) * 100
    )

In [132]:
WIDTH = 100
HEIGHT = 100
NUM_AGENTS = 1
MAX_STEPS = 100000
DIRTY_CELLS = 0.8

start_time = time.time()
result = run_model(WIDTH, HEIGHT, NUM_AGENTS, MAX_STEPS, DIRTY_CELLS)
total_time = time.time() - start_time

print("Tiempo total de la ejecución: ", total_time)
print("Número de pasos realizados: " + str(result[0]))
print("Porcentaje de celdas limpias: " + str(result[1]) + "%")

Tiempo total de la ejecución:  1.9473888874053955
Número de pasos realizados: 100000
Porcentaje de celdas limpias: 90.42999999999999%
